## Importación

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTEN

In [26]:
df=pd.read_csv("https://raw.githubusercontent.com/nathygonzalez/PR06-Machine-Learning-Group-Project/develop/data/df_cleaned.csv")

C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (99) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## depuración columna "gk"

In [27]:
df['gk'] =df['gk'].replace("+","")

In [28]:
df['gk'] = df['gk'].str.replace(r'\+', '', regex=True)


In [29]:
df.gk.fillna(0,inplace=True)

In [30]:
df['gk']=df.gk.astype(int)

In [31]:
df['gk']

0        15
1        16
2        14
3        17
4        87
         ..
87742     0
87743     0
87744     0
87745     0
87746     0
Name: gk, Length: 87747, dtype: int32

## Añadir variable años pendientes de contrato

In [32]:
df['años_de_contrato']=np.where((df['club_contract_valid_until']-df['año_version'])<0,0,(df['club_contract_valid_until']-df['año_version']))

In [33]:
#comprobación
df.groupby('años_de_contrato')['años_de_contrato'].count()

años_de_contrato
0.0    30707
1.0    21182
2.0    17779
3.0     9287
4.0     4872
5.0     2502
6.0     1413
7.0        3
8.0        2
Name: años_de_contrato, dtype: int64

## Eliminación de ruido

In [34]:
#elimino columnas (según Sergio)
'''df.drop(['sofifa_id',
 'short_name',
 'long_name',
 'player_positions',
     'dob', 'club_team_id',
 'club_name',
 'league_name',
 'league_level',
'club_jersey_number',
 'club_joined',
 'club_contract_valid_until',
 'nationality_id',
 'nationality_name',
 'nation_team_id','work_rate',
 'body_type',
 'real_face',
    'player_traits'],axis=1,inplace=True)'''

"df.drop(['sofifa_id',\n 'short_name',\n 'long_name',\n 'player_positions',\n     'dob', 'club_team_id',\n 'club_name',\n 'league_name',\n 'league_level',\n'club_jersey_number',\n 'club_joined',\n 'club_contract_valid_until',\n 'nationality_id',\n 'nationality_name',\n 'nation_team_id','work_rate',\n 'body_type',\n 'real_face',\n    'player_traits'],axis=1,inplace=True)"

In [35]:
#eliminamos columnas que sabemos seguro que sólo aportan ruido (según Carlos)
df.drop(["sofifa_id",'short_name', 'long_name',"nationality_id","dob",'club_team_id', 'club_name', 'league_name',
        'club_jersey_number', 'club_joined', 'club_contract_valid_until','nationality_id', 'nationality_name','player_traits',
        'ranking','puntos','año_version'],axis=1,inplace=True)

## Agrupar columna bodytype

In [36]:
Lean=['Lean (170-)','Lean (185+)','Lean (170-185)']
Normal=['Normal (170-)','Normal (170-185)','Normal (185+)']
Stocky=['Stocky (170-)','Stocky (170-185)','Stocky (185+)']

#agrupo posiciones en posiciones genéricas    
for i in df.index:
    if df.at[i,'body_type'] in Lean:
        df.at[i,'body_type']='Lean'
    elif df.at[i,'body_type'] in Normal:
        df.at[i,'body_type']='Normal'
    elif df.at[i,'body_type'] in Stocky:
        df.at[i,'body_type']='Stocky'
    else:
        pass

## Exploración feature engineering

In [37]:
df.preferred_foot.value_counts() #categorica a OHE

Right    66970
Left     20777
Name: preferred_foot, dtype: int64

In [38]:
df.work_rate.value_counts() #categorica transformarla con un ordinal encoder

Medium/Medium    47248
High/Medium      15911
Medium/High       8196
High/High         4712
Medium/Low        4072
High/Low          3332
Low/Medium        2078
Low/High          2058
Low/Low            140
Name: work_rate, dtype: int64

In [39]:
df.body_type.value_counts() #categorica a OHE

Normal    51581
Lean      30898
Stocky     5099
Unique      169
Name: body_type, dtype: int64

In [40]:
df.real_face.value_counts() #categorica a OHE

No     78885
Yes     8862
Name: real_face, dtype: int64

## Feature engineering (transformar todas las variables a numéricas menos player_positions)

In [43]:
#preparación para transformación categorica a num
work_rate_ord=['Low/Low','Low/Medium','Low/High','Medium/Low','Medium/Medium','Medium/High','High/Low','High/Medium','High/High']
league_level_ord=['0.0','4.0','3.0','2.0','1.0']
df['league_level']=df['league_level'].apply(str)

In [51]:
#Ordinal Encoder
df[['work_rate','league_level']]=OrdinalEncoder(categories=[work_rate_ord,league_level_ord]).fit_transform(df[['work_rate','league_level']])

In [58]:
#OHE
df[['preferred_foot','body_type_1','body_type_2','body_type_3','real_face']]=OneHotEncoder(drop='first',sparse=False).fit_transform(df[['preferred_foot','body_type','real_face']])                                          

In [59]:
#comprobar columnas
categorical = df.dtypes == object
categorical[categorical==True]

player_positions    True
body_type           True
dtype: bool

In [60]:
#eliminar columna body_type
df.drop(["body_type"],axis=1,inplace=True)

## Filtramos el dataset por posicion

In [61]:
#filtramos dataset para evitar excesiva coliniaridad
df=df[df['player_positions']=='portero']
#eliminar columna player_positions
df.drop(["player_positions"],axis=1,inplace=True)

C:\Users\carlo\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [63]:
df.head()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_position,nation_team_id,...,gk,progresion_anual,overall_team/year,teams_in_league/year,mean_league/year,mean_team/year,años_de_contrato,body_type_1,body_type_2,body_type_3
4,90,90,63500000.0,300000.0,28,193,92,4.0,1,1,...,87,0.0,2061.0,18.0,71.280514,79.269231,4.0,1.0,0.0,0.0
29,86,90,27000000.0,170000.0,22,199,88,4.0,1,1,...,85,0.0,2305.0,20.0,71.277778,74.354839,4.0,0.0,0.0,0.0
31,85,85,18000000.0,170000.0,32,196,90,4.0,0,1,...,83,0.0,2305.0,20.0,71.277778,74.354839,1.0,1.0,0.0,0.0
34,85,86,21000000.0,160000.0,27,188,78,4.0,1,1,...,84,0.0,1991.0,20.0,71.277778,73.740741,4.0,0.0,0.0,0.0
44,84,84,16500000.0,140000.0,33,185,84,4.0,1,1,...,82,0.0,2418.0,20.0,71.357143,75.562500,2.0,1.0,0.0,0.0


## Features selection

In [67]:
target=[]
#elegimos el % de target para que sea 0 y 1
for i in df.progresion_anual:
    if i>0.1:
        target.append(1)
    else:
        target.append(0)
df["target"]=target

<ipython-input-67-40676595d978>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"]=target


In [68]:
X=df.drop(["progresion_anual","target"],axis=1)
y=df.target

In [70]:
#seleccion de columnas 
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X,y)

SelectFromModel(estimator=RandomForestClassifier())

In [71]:
#columnas que coge
sel.get_support()

array([ True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
       False, False, False])

In [72]:
selected_feat= X.columns[(sel.get_support())]
len(selected_feat) #selecciona 42 columnas

42

In [78]:
#vemos que variables son
selected_feat

Index(['overall', 'potential', 'value_eur', 'wage_eur', 'age', 'height_cm',
       'weight_kg', 'attacking_crossing', 'attacking_short_passing',
       'attacking_volleys', 'skill_curve', 'skill_fk_accuracy',
       'skill_long_passing', 'skill_ball_control', 'movement_acceleration',
       'movement_sprint_speed', 'movement_agility', 'movement_reactions',
       'movement_balance', 'power_shot_power', 'power_jumping',
       'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed',
       'gk', 'overall_team/year', 'teams_in_league/year', 'mean_league/year',
       'mean_team/year'],
      dtype='object')

In [76]:
#redefinimos X:
X=df[selected_feat.to_list()]

## Primer modelo naive

In [79]:
#split el dataframe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [80]:
model=RandomForestClassifier()

In [81]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [82]:
model.score(X_test,y_test)

0.9574468085106383

In [83]:
y_pred=model.predict(X_test)

In [86]:
print(classification_report(y_test,y_pred)) 
#Se observa accury muy alta => consecuencia posible de df muy desbalanceado

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1845
           1       0.00      0.00      0.00        82

    accuracy                           0.96      1927
   macro avg       0.48      0.50      0.49      1927
weighted avg       0.92      0.96      0.94      1927



C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [98]:
#desbalanceo de X
1845/1927

0.9574468085106383

## Segundo modelo con undersampling

In [100]:
rus = RandomUnderSampler(random_state=0)
X_res, y_res =rus.fit_resample(X, y)

#split el dataframe
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=0)



In [101]:
model=RandomForestClassifier()
model.fit(X_train,y_train)

y_pred=model.predict(X_test)

print(classification_report(y_test, y_pred))
#random forest classifier consigue un 73% de accuracy

RandomForestClassifier()

In [105]:
model2=SVC()
model2.fit(X_train,y_train)
y_pred=model2.predict(X_test)
print(classification_report(y_test, y_pred))
#SVC consigue un 66% de accuracy

              precision    recall  f1-score   support

           0       0.72      0.42      0.54        80
           1       0.63      0.86      0.73        93

    accuracy                           0.66       173
   macro avg       0.68      0.64      0.63       173
weighted avg       0.68      0.66      0.64       173



## Tercer modelo con Over-sampling

In [107]:
res= SMOTEN()

In [109]:
X_res,y_res=res.fit_resample(X, y)

In [112]:
#comprobación
sum(y_res)/len(y_res) 

0.5

In [113]:
#split el dataframe
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=0)

In [114]:
model3=RandomForestClassifier()
model3.fit(X_train,y_train)
y_pred=model3.predict(X_test)
print(classification_report(y_test, y_pred))
#random forest classifier con oversampling consigue un 97% de accuracy

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1847
           1       0.99      0.96      0.97      1834

    accuracy                           0.97      3681
   macro avg       0.97      0.97      0.97      3681
weighted avg       0.97      0.97      0.97      3681



In [115]:
model4=SVC()
model4.fit(X_train,y_train).score(X_test,y_test)
y_pred=model4.predict(X_test)
print(classification_report(y_test, y_pred))
#SVC con oversampling consigue un 77% de accuracy

              precision    recall  f1-score   support

           0       0.86      0.65      0.74      1847
           1       0.72      0.89      0.80      1834

    accuracy                           0.77      3681
   macro avg       0.79      0.77      0.77      3681
weighted avg       0.79      0.77      0.77      3681



## Parameter tunnig

In [125]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [127]:
%%time
from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=model3, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

Wall time: 18min 10s


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [200, 500]})

In [128]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 500}

In [129]:
model_def=RandomForestClassifier(max_features='auto', n_estimators= 500, max_depth=8, criterion='entropy')
model_def.fit(X_train,y_train)
y_pred=model_def.predict(X_test)
print(classification_report(y_test, y_pred))
#random forest classifier con oversampling consigue un 96% de accuracy

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1847
           1       0.96      0.96      0.96      1834

    accuracy                           0.96      3681
   macro avg       0.96      0.96      0.96      3681
weighted avg       0.96      0.96      0.96      3681



## Mirar la previsión de jugadores de fifa22

In [184]:
df22=pd.read_csv('data/players_22.csv')
#eliminamos columnas de url que no aportan información
df22 = df22[df22.columns.drop(list(df22.filter(regex='url')))]


C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [185]:
#funcion transformación en lista
def lista_posicion(x):
    return (x.replace(' ','').split(','))

In [186]:
#transforma valores de columna player_positions en una lista
df22['player_positions']=df22['player_positions'].apply(lambda x: lista_posicion(x))

In [187]:
#defino listas para agrupar posiciones por 4 categorias
pos_defensas=['LCB','RCB','CB','RB','LB','RWB','LWB']
pos_medios=['LW','LCM','RCM','CAM','RW','LDM','CDM','CM','RDM','RM','RAM','LAM','LM']
pos_delanteros=['CF','ST','RS','LS','RF']

In [188]:
#agrupo posiciones en posiciones genéricas    
for i in df22.index:
            if len(df22.at[i,'player_positions'])==1:
                if df22.at[i,'player_positions'][0]=='GK':
                    df22.at[i,'player_positions']='portero'
                if df22.at[i,'player_positions'][0] in pos_defensas:
                    df22.at[i,'player_positions']='defensa'
                elif df22.at[i,'player_positions'][0] in pos_medios:
                    df22.at[i,'player_positions']='centrocampista'
                elif df22.at[i,'player_positions'][0] in pos_delanteros:
                    df22.at[i,'player_positions']='delantero'
            else:
                if df22.at[i,'player_positions'][0] in pos_defensas:
                    df22.at[i,'player_positions']='defensa'
                elif df22.at[i,'player_positions'][0] in pos_medios:
                    df22.at[i,'player_positions']='centrocampista'
                elif df22.at[i,'player_positions'][0] in pos_delanteros:
                    df22.at[i,'player_positions']='delantero'
                elif df22.at[i,'player_positions'][0]=='GK':
                    df22.at[i,'player_positions']='portero'

In [189]:
#filtramos dataset 
df22=df22[df22['player_positions']=='portero']

In [190]:
#Agrupa por Equipo y año versión(temporada) y añade el total en cada jugador
df22['overall_team/year'] = df22.groupby('club_team_id')['overall'].transform('sum')

In [191]:
#Crea columna nº de equipos por liga/año
df22['teams_in_league/year'] = df22.groupby('league_name')['club_team_id'].transform('nunique')

In [192]:
#Crea columna media liga/año
df22['mean_league/year'] = df22.groupby('league_name')['overall'].transform('mean')

In [193]:
#Crea columna media equipo/año
df22['mean_team/year'] = df22.groupby('club_team_id')['overall'].transform('mean')

In [194]:
#df solo con las columnas a transformar
posiciones=df22[["ls","st","rs","lw","lf","cf","rf","rw", 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm',
     'lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb',"gk"]]

In [198]:
df22['gk'] = df22['gk'].str.replace(r'\+', ' ', regex=True)

In [200]:
#funcion para limpiar valores
def limp_posiciones(x):
    return x.split(" ")[0]

In [201]:
#limpiar en df
for i in posiciones.columns:
    df22[i]=df22[i].astype(str)
    df22[i]=df22[i].apply(limp_posiciones)

In [203]:
df22.gk.fillna(0,inplace=True)

In [204]:
df22['gk']=df22.gk.astype(int)

In [205]:
df22['gk']

5        89
7        88
8        88
12       86
18       87
         ..
19210    47
19216    47
19225    46
19227    46
19231    46
Name: gk, Length: 2132, dtype: int32

In [207]:
X22=df22[selected_feat.to_list()]

In [210]:
X22.fillna(0,inplace=True)

C:\Users\carlo\anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [211]:
y_pred=model_def.predict(X22)

In [219]:
ind_list=X22[y_pred==1].index.to_list()

In [228]:
#top 5 porteros con mayor proyección
df22[df22.index.isin(ind_list)].sort_values(by='overall', ascending=False).head()

,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,...,lb,lcb,cb,rcb,rb,gk,overall_team/year,teams_in_league/year,mean_league/year,mean_team/year
15058,262254,P. Bos,Pieter Bos,portero,61,65,375000.0,950.0,24,1997-02-23,...,18+2,21+2,21+2,21+2,18+2,60,184.0,18.0,64.537313,61.333333
15438,246246,B. Leneis,Benjamin Leneis,portero,60,70,475000.0,2000.0,22,1999-03-08,...,16+2,17+2,17+2,17+2,16+2,59,179.0,20.0,60.508475,59.666667
15731,259783,S. Rojas,Sebastián André Rojas Quesada,portero,60,65,300000.0,500.0,21,1999-10-21,...,18+2,20+2,20+2,20+2,18+2,59,245.0,6.0,62.857143,61.250000
15400,243737,M. Kamenz,Matti Kamenz,portero,60,66,325000.0,500.0,22,1998-08-09,...,17+2,17+2,17+2,17+2,17+2,59,185.0,20.0,60.508475,61.666667
15955,188815,R. Lejeune,Romain Lejeune,portero,59,59,150000.0,800.0,30,1991-05-21,...,20+2,21+2,21+2,21+2,20+2,58,182.0,20.0,63.912281,60.666667
